In [ ]:
#in the absence of known webapi which can be used to retrieve the data , I am using web scrapping
#to get the corporate bond info. getting it from a webapi which will return the bond yields will be much 
#easier

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import asyncio
import aiohttp
import pandas as pd
import nest_asyncio

pd.set_option('display.max_colwidth', 200)

# URL of the source
url = 'https://www.hl.co.uk/shares/corporate-bonds-gilts/bond-prices/gbp-bonds'

# Load data from URL
tables = pd.read_html(url)
# Extract the table of interest (in this case, the first table on the page)
df = tables[0]
df=df.drop(columns=['Actions'])
df.head(10)



In [ ]:
df_issuer = df
print(df_issuer['Issuer'])
# Split "issuer" column by "|" character
df_issuer[['Name', 'ISIN', 'SEDOL']] = df_issuer['Issuer'].str.split('|', expand=True)

# Remove leading/trailing whitespace from the newly created columns
df_issuer['Name'] = df_issuer['Name'].str.strip()
df_issuer['ISIN'] = df_issuer['ISIN'].str.strip()
df_issuer['SEDOL'] = df_issuer['SEDOL'].str.strip()

# Drop the original "issuer" column
df_issuer.drop('Issuer', axis=1, inplace=True)

# Get a list of all column names except "Name", "ISIN", and "SEDOL"
other_columns = [col for col in df_issuer.columns if col not in ['Name', 'ISIN', 'SEDOL']]

# Reorder columns with "Name", "ISIN", and "SEDOL" as the first columns, followed by the remaining columns
df_issuer = df_issuer[['Name']+other_columns+['ISIN','SEDOL']]

# Display the updated DataFrame with additional columns
print(df_issuer)


In [ ]:
bond_underlyings=df_issuer['SEDOL'].values
bond_underlyings

In [ ]:
%%capture
%run gilts_yield_calculator.ipynb

In [ ]:
def get_coupon_frequancy(frequency):
    switch_dict = {
        "Annually": 1,
        "Bi-annually": 2,
        "Quarterly": 4
    }
    return switch_dict.get(frequency, None)


In [ ]:
# Define a function to generate link HTML
def generate_link(url):
    return f'<a href="{url}" target="_blank">{url}</a>'



In [ ]:
b_df = pd.DataFrame(columns = ['SEDOL', 'CouponFreq'])



In [ ]:
def add_bond_to_data_frame(bond_underlying: str, concate_to_df):
    print('trying to scrap bond data {} from url '.format(bond_underlying))
    url_hl_search = f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}'
    df = pd.read_html(url_hl_search)[0]
    df = df.T
    df.reset_index(drop=True, inplace=True)
    df.columns = df.iloc[0].str.rstrip(':')
    df.drop(0, inplace=True)
    df['Coupon_Freq'] = df['Coupon frequency'].map(get_coupon_frequancy).astype('Int64')
    df['SEDOL'] = bond_underlying
    df = df[['Coupon_Freq', 'SEDOL']]
    concate_to_df = pd.concat([concate_to_df, df], ignore_index=False)
    return concate_to_df

In [ ]:
bond_underlying = 'B03HC45'
#a = [add_bond_to_data_frame(b,b_df) for b in bond_underlyings]
#b_df=add_bond_to_data_frame(bond_underlying,b_df)
#b_df=b_df.style.format({'URL': generate_link})
#b_df



In [ ]:
get_bond_page= (lambda bond_underlying : 
                   f'https://www.hl.co.uk/shares/shares-search-results/{bond_underlying}')

In [ ]:

nest_asyncio.apply()  # Run this line if using Jupyter Notebook
async def fetch_data(bond_underlying: str):
    async with aiohttp.ClientSession() as session:
        url_hl_search = get_bond_page(bond_underlying)           
        async with session.get(url_hl_search) as response:
            html = await response.text()
            df= pd.read_html(html,flavor='bs4')[0]
            df = df.transpose()
            df.reset_index(drop=True, inplace=True)
            df.columns = df.iloc[0].str.rstrip(':')
            df.drop(0, inplace=True)
            df['CouponFreq'] = df['Coupon frequency'].map(get_coupon_frequancy).astype('Int64')
            df['SEDOL'] = bond_underlying
            df = df[['CouponFreq', 'SEDOL']]
            return df

async def get_bond_coupon_freq(bond_underlying: str):
    try:
        print('trying to scrape bond data {} from url '.format(bond_underlying))
        df = await fetch_data(bond_underlying)
        return df
    except Exception as e:        
        print("error when calling bond {} m exception \t {}".format(bond_underlying,e))

async def main():
    global b_df
    tasks = [get_bond_coupon_freq(bond_underlying) for bond_underlying in bond_underlyings]
    results = await asyncio.gather(*tasks)
    b_df= pd.concat([b_df]+results)

await main()


In [ ]:
b_df

In [ ]:
df_corp_bonds = pd.merge(df_issuer,b_df, on='SEDOL', how='outer')


In [ ]:
df_corp_bonds=df_corp_bonds.rename(columns={'Coupon (%)': 'Coupon'})
df_corp_bonds=df_corp_bonds.dropna()
df_corp_bonds

In [ ]:
date_format ="%d %B %Y"
df_corp_bonds['Maturity'] = df_corp_bonds['Maturity'].apply(lambda maturity:pd.to_datetime(maturity).date())

df_corp_bonds['Ttm'] = df_corp_bonds["Maturity"].apply(lambda maturity: calculate_time_to_maturity(maturity))
df_corp_bonds['NextCouponDate'] = df_corp_bonds.apply(lambda row: get_next_coupon_date(row['Maturity'],row['CouponFreq']),axis=1)

cols = ['Maturity','NextCouponDate']
for col in cols:
    df_corp_bonds[col]= df_corp_bonds[col].apply(lambda date: pd.to_datetime(date).date())


In [ ]:
COUPON_EX_DATE_OFFSET=2 # how many business dates prior to date offset the Ex date is

df_corp_bonds['AccruedInterest']=df_corp_bonds.apply(lambda x: calculate_accrued_interest(coupon=x['Coupon'],
                                                                                          face_value=FACE_VALUE,
                                                                                          years_to_maturity=x['Ttm'],
                                                                                          payment_frequency=x['CouponFreq'],
                                                                                          next_coupon_date=x['NextCouponDate'],
                                                                                          coupon_ex_date_offset=COUPON_EX_DATE_OFFSET),axis=1)
df_corp_bonds['DirtyPrice']=df_corp_bonds.apply(lambda x: calculate_dirty_price(x['Price'],x['AccruedInterest']),axis=1)
df_corp_bonds['DirtyYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['DirtyPrice'],x['CouponFreq']), axis=1)
df_corp_bonds['CleanYield'] = df_corp_bonds.apply(lambda x: calculate_ytm(x['Coupon'], FACE_VALUE, x['Ttm'],x['Price'],x['CouponFreq']), axis=1)

In [ ]:

df_less_than_Ttm = df_corp_bonds[df_corp_bonds['Ttm'] < 5]
df_less_than_Ttm=df_less_than_Ttm.sort_values("DirtyYield", ascending=False)
df_less_than_Ttm['URL']=df_less_than_Ttm['SEDOL'].apply(lambda sedol: get_bond_page(sedol))
cols = ['Name','URL','SEDOL','Coupon', 'Maturity', 'Price', 'Ttm','DirtyPrice', 'DirtyYield', 'CleanYield','CouponFreq','NextCouponDate']
df_less_than_Ttm = df_less_than_Ttm[cols]
df_less_than_Ttm.style.format({'URL': generate_link})
df_less_than_Ttm.sort_values(by='DirtyYield',ascending=False, inplace=True)

In [ ]:
df_less_than_Ttm